## Simulation of fields penetrating two slabs with normal incidence
### Colin Renard (50012000) & Mathieu Reniers (30322000)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib qt

#### Parameters

In [2]:
Nf = 50
fc = 3e9         
df = 1e9                                # Deviation frequency [Hz]
B = (fc+df) - (fc-df)                   # Bandwidth [Hz]
f = np.linspace(fc-df,fc+df,Nf)         # Frequency axis [Hz]
w,wc = 2*np.pi*f, 2*np.pi*fc            # Pulsation axis & central pulsation
sigma_w = 2*np.pi*0.2*df                # Standard deviation of pulsations

A = (1/(np.sqrt(2*np.pi)*sigma_w)) * np.exp(-(w-wc)**2/(2*(sigma_w**2)))        # Amplitude distribution over frequences (Gaussian)
#A = A*np.sqrt(4*sigma_w*(np.pi**(3/2)))  
#A = np.array([0,0,2,0,0])


# Vacuum
n0 = 1; e0 = 8.85e-12; mu0 = 4*np.pi*1e-7
c = 1/np.sqrt(e0*mu0)
 
# Media

a = 1.5                 # Scaling factor

l0 = 1.5*a              # Length of medium 0 [m]
k0 = w * n0 /c          # Wavenumber 0

n1 = 1.2                # Refractive index of medium 1
l1 = 1*a                # Length of medium 1 [m]
k1 = w * n1 /c          # Wavenumber 1

n2 = 1.7                # Refractive index of medium 2
l2 = 1.5*a              # Length of medium 2 [m]
k2 = w * n2 /c          # Wavenumber 2

n3 = n0                 # Refractive index of medium 3
l3 = 1*a              # Length of medium 3 [m]
k3 = w * n3 /c          # Wavenumber 3

# Axis
Nz = 100           # Precision
z0 = np.linspace(0,l0, int(l0*Nz))
z1 = np.linspace(0,l1, int(l1*Nz))
z2 = np.linspace(0,l2, int(l2*Nz))
z3 = np.linspace(0,l3, int(l3*Nz))

z = np.array([*z0, *z1 + l0, *z2 + (l0+l1), *z3 + (l0+l1+l2)])


In [3]:
# Functions

rho     =   lambda n,np : (n-np)/(n+np)                                 # Return rho of interface n|np
tau     =   lambda n,np : (2*n)/(n+np)                                  # Return tau of interface n|np
gamma   =   lambda gammap, rho : (rho + gammap) / (1 + rho*gammap)      # Return gamma of interface rho: _ | gammap

def interface_R2L(Ep_,Em_,n,np):
    rho_i = rho(n,np)
    tau_i = tau(n,np)
    Ep = (1/tau_i) * (1 * Ep_ + rho_i * Em_)
    Em = (1/tau_i) * (rho_i * Ep_ + 1 * Em_)
    return Ep,Em

def propagation_R2L(Ep2,Em2,k,l):
    Ep1 = np.exp(1j*k*l) * Ep2
    Em1 = np.exp(-1j*k*l) * Em2
    return Ep1,Em1

def propagation_L2R(Ep1,Em1,k,l):
    Ep2 = np.exp(-1j*k*l) * Ep1
    Em2 = np.exp(1j*k*l) * Em1
    return Ep2,Em2


In [4]:
t = np.linspace(0,1e-6,1000)

E = np.zeros(len(z))
E_time = np.zeros((len(t),len(E)))

for i in range(len(t)):

    # Superposition principle
    for j in range(len(f)):

        E3p,E3m = propagation_L2R(1,0,k3[j],z3)                        # Waves in (3)
        E3p_l, E3m_l = interface_R2L(E3p[0],E3m[0],n2,n3)           # Left value at interface III
        E2p, E2m = propagation_R2L(E3p_l,E3m_l, k2[j], np.flip(z2))    # Waves in (2)
        E2p_l, E2m_l = interface_R2L(E2p[0],E2m[0],n1,n2)           # Left value at interface II
        E1p, E1m = propagation_R2L(E2p_l,E2m_l, k1[j], np.flip(z1))    # Waves in (1)
        E1p_l, E1m_l = interface_R2L(E1p[0],E1m[0],n0,n1)           # Left value at interface I
        E0p, E0m = propagation_R2L(E1p_l,E1m_l, k0[j], np.flip(z0))    # Waves in (0)

        E = np.array([*(E0p+E0m) , *(E1p+E1m) , *(E2p+E2m) , *(E3p+E3m) ])
    
        E_time[i] += A[j]*np.real(E*np.exp(1j*w[j]*t[i]))

E_time /= np.max(E_time)

In [8]:
c = ["tab:orange", "tab:green", "tab:red", "tab:blue"]

fig, ax = plt.subplots()

ax.set_xlabel("z [m]")
ax.set_ylabel("Electric Field [V/m]")
ax.grid()

line, = ax.plot(z, E_time[0])
yrange = plt.gca().get_ylim()


plt.fill_between(z0, yrange[0], yrange[1], color=c[1],alpha=0.3, label=r"$n=1$")
plt.fill_between(l0+z1, yrange[0], yrange[1], color=c[0],alpha=0.3, label=r"$n=$"+str(n1))
plt.fill_between(l0+l1+z2, yrange[0], yrange[1], color=c[2],alpha=0.3, label=r"$n=$"+str(n2))
plt.fill_between(l0+l1+l2+z3, yrange[0], yrange[1], color=c[1],alpha=0.3)
ax.legend(loc="upper right")

def anim(i):       
    #line.set_ydata(compute_s(z,t[i]))  # update the data
    line.set_ydata(E_time[i])
    return line,

ani = animation.FuncAnimation(fig, anim, frames=len(t),interval=100, repeat=True ,blit=False)
plt.show()

#writer = animation.PillowWriter(fps=15, metadata=dict(artist='Me'),bitrate=1800)
#ani.save('layer_paquet_wave_1f.gif', writer=writer)